In [3]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_1_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_1_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_1.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_1_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_1_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_1.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_1_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_1_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_1_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_1_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_1_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_1_features.csv')

For BBBP (Classification) \
Valid 2 \
No missing in chemberta2 and molformer \
3 missing in molbert \
Test \
no missing in chemberta2, 1 missing in molformer, 2 missing in molbert





In [4]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8731707317073171,
  'F1 Score': 0.9133333333333333,
  'ROC-AUC': 0.9536774193548387,
  'PR-AUC': 0.9855666168892795},
 'Molformer': {'Accuracy': 0.8823529411764706,
  'F1 Score': 0.9235668789808917,
  'ROC-AUC': 0.8681168831168831,
  'PR-AUC': 0.9131281577368526},
 'Molbert': {'Accuracy': 0.8916256157635468,
  'F1 Score': 0.930379746835443,
  'ROC-AUC': 0.9509011396766499,
  'PR-AUC': 0.9829668845173022}}

In [5]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_smiles_valid2 = list(set(missing_smiles_molformer_valid2).union(set(missing_smiles_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_smiles_valid2}")

Missing smiles in molformer_valid2: set()
Missing smiles in molbert_valid2: {'[Na+].[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](C([O-])=O)c3cscc3)C(=O)N2[C@H]1C([O-])=O', '[Na+].[Na+].[Na+].[O-]C(=O)[P]([O-])([O-])=O', '[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3'}
These indices will be removed from the valid2 set: ['[Na+].[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](C([O-])=O)c3cscc3)C(=O)N2[C@H]1C([O-])=O', '[Na+].[Na+].[Na+].[O-]C(=O)[P]([O-])([O-])=O', '[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3']


In [6]:
# Function to remove invalid SMILES
def remove_invalid_smiles(df, invalid_smiles_list):
    return df[~df['smiles'].isin(invalid_smiles_list)]

# Remove invalid SMILES from each dataframe
bbbp_chemberta2_valid2 = remove_invalid_smiles(bbbp_chemberta2_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_valid2 = remove_invalid_smiles(bbbp_molformer_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_valid2 = remove_invalid_smiles(bbbp_molbert_valid2, combined_invalid_smiles_valid2)
bbbp_chemberta2_features_valid2 = remove_invalid_smiles(bbbp_chemberta2_features_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_features_valid2 = remove_invalid_smiles(bbbp_molformer_features_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_features_valid2 = remove_invalid_smiles(bbbp_molbert_features_valid2, combined_invalid_smiles_valid2)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(407, 8)
(407, 6)
(407, 4)
(407, 386)
(407, 769)
(407, 769)


In [7]:
bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

# Convert the ensemble target to a Series if not already done
bbbp_y_ensemble_valid2_s = pd.Series(bbbp_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)

bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)

bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

# bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# Function to calculate BCE for each row
def calculate_bce_rowwise(y_true, y_pred):
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
bbbp_X_ensemble_valid2 = pd.concat([bce_loss_df, train_bbbp_prob], axis=1)

In [8]:
import numpy as np
import pandas as pd
from skglm import GroupLasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([bbbp_chemberta2_prob, bbbp_chemberta2_features], axis=1)
molformer_X = pd.concat([bbbp_molformer_prob, bbbp_molformer_features], axis=1)
molbert_X = pd.concat([bbbp_molbert_prob, bbbp_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

In [9]:
import pandas as pd

# Export valid2 probabilities
bbbp_chemberta2_prob.to_csv('./processed_data/bbbp_chemberta_prob.csv', index=False)
bbbp_molformer_prob.to_csv('./processed_data/bbbp_molformer_prob.csv', index=False)
bbbp_molbert_prob.to_csv('./processed_data/bbbp_molbert_prob.csv', index=False)

# Convert numpy arrays to pandas DataFrames and export
pd.DataFrame(chemberta_X_scaled).to_csv('./processed_data/bbbp_chemberta_X.csv', index=False)
pd.DataFrame(molformer_X_scaled).to_csv('./processed_data/bbbp_molformer_X.csv', index=False)
pd.DataFrame(molbert_X_scaled).to_csv('./processed_data/bbbp_molbert_X.csv', index=False)

# Export BCE targets
chemberta_y_bce.to_csv('./processed_data/bbbp_chemberta_y.csv', index=False)
molformer_y_bce.to_csv('./processed_data/bbbp_molformer_y.csv', index=False)
molbert_y_bce.to_csv('./processed_data/bbbp_molbert_y.csv', index=False)

# Export ensemble predictions
bbbp_y_ensemble_valid2.to_csv('./processed_data/bbbp_y_ensemble_valid2.csv', index=False)

In [10]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")

# Combine the invalid smiles from molformer_test with the missing smiles from molbert_test
combined_invalid_smiles_test = list(set(missing_smiles_molformer_test).union(set(missing_smiles_molbert_test)))
print(f"These indices will be removed from the test set: {combined_invalid_smiles_test}")

Missing smiles in molformer_test: {'s1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N'}
Missing smiles in molbert_test: {'s1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', '[Na+].C[C@@H](O)[C@@H]1[C@H]2SC(=C(N2C1=O)C([O-])=O)COC(N)=O'}
These indices will be removed from the test set: ['s1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', '[Na+].C[C@@H](O)[C@@H]1[C@H]2SC(=C(N2C1=O)C([O-])=O)COC(N)=O']


In [11]:
# Remove invalid SMILES from each dataframe
bbbp_chemberta2_test = remove_invalid_smiles(bbbp_chemberta2_test, combined_invalid_smiles_test)
bbbp_molformer_test = remove_invalid_smiles(bbbp_molformer_test, combined_invalid_smiles_test)
bbbp_molbert_test = remove_invalid_smiles(bbbp_molbert_test, combined_invalid_smiles_test)
bbbp_chemberta2_features_test = remove_invalid_smiles(bbbp_chemberta2_features_test, combined_invalid_smiles_test)
bbbp_molformer_features_test = remove_invalid_smiles(bbbp_molformer_features_test, combined_invalid_smiles_test)
bbbp_molbert_features_test = remove_invalid_smiles(bbbp_molbert_features_test, combined_invalid_smiles_test)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(203, 8)
(203, 6)
(203, 4)
(203, 386)
(203, 769)
(203, 769)


In [12]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
bbbp_chemberta2_prob_test = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob_test.reset_index(drop=True, inplace=True)

bbbp_molformer_prob_test = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob_test.reset_index(drop=True, inplace=True)

bbbp_molbert_prob_test = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob_test.reset_index(drop=True, inplace=True)

bbbp_chemberta2_features_t = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features_t.reset_index(drop=True, inplace=True)

bbbp_molformer_features_t  = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features_t.reset_index(drop=True, inplace=True)

bbbp_molbert_features_t = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([bbbp_chemberta2_prob_test, bbbp_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([bbbp_molformer_prob_test, bbbp_molformer_features_t], axis=1)
molbert_X_test = pd.concat([bbbp_molbert_prob_test, bbbp_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

In [13]:
import pandas as pd

# Export test probabilities
bbbp_chemberta2_prob_test.to_csv('./processed_data/bbbp_chemberta_prob_test.csv', index=False)
bbbp_molformer_prob_test.to_csv('./processed_data/bbbp_molformer_prob_test.csv', index=False)
bbbp_molbert_prob_test.to_csv('./processed_data/bbbp_molbert_prob_test.csv', index=False)

# Convert numpy arrays to pandas DataFrames and export test scaled data
pd.DataFrame(chemberta_X_test_scaled).to_csv('./processed_data/bbbp_chemberta_X_test.csv', index=False)
pd.DataFrame(molformer_X_test_scaled).to_csv('./processed_data/bbbp_molformer_X_test.csv', index=False)
pd.DataFrame(molbert_X_test_scaled).to_csv('./processed_data/bbbp_molbert_X_test.csv', index=False)

# Export ensemble test class labels
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np']
bbbp_y_ensemble_test.to_csv('./processed_data/bbbp_y_ensemble_test.csv', index=False)